# Classify Stars in Colliding Galaxies

Using ~5000 labeled stars out of ~80,000 in each galaxy (170,000 stars total), make a model to predict and/or classify all the stars in the collection of the galaxy collision.

![KNN Classifier results for colliding galaxies](Assets/KNNPredictionResult.png "Classifying Stars in Colliding Galaxies")


As an optional assignment: 
- Find all neighboring pairs of points in two approaching galaxies.

Follow the creation of two synthetically created star locations in Euclidean coordinates. The galaxies are set to collide with each other. Techniques described are an approach to classifying which stars belong to which galaxy after the collision. We also later identify the coordinates of pairs of points that lie within a radius R of each other. Points already lying with radius R in either galaxy are ignored, and the focus is narrowed to points in one galaxy in close proximity to the points in the other galaxy.

We synthesize ~ 80,000 stars in each of two galaxies—a fictitious GFFA ("galaxy far, far away") and a fictitious "Xeno", which is the backdrop for Superman's Krypton. Combined, this is over 100,000 stars, for which we search for neighboring pairs on the order of a few light years apart.

Play the following video which plots and rotates the combined galaxies. It plots a 3D scatter plot of the nearly 200,000 stars in the two galaxies. It gradually reduces the opacity of each star to ultimately reveal the one or two or handfuls of stars that are within a given radius of each other as a result of the collision of the two galaxies. The center bulge in each galaxy is so dense that the red zone stars are not visible until we turn the opacity of the stars down significantly.


<video controls src="Videos/CollidingGalaxiesAnimation.mp4" />



# Learning Objectives:
 
1. Apply Multiple Classification Algorithms with GPU to classify stars belonging to each galaxy within a combined super galaxy to determine most accurate model.
1. Apply Intel® Extension for Scikit-learn* patch and SYCL context to compute on available GPU resource.
1. **Optional**: Use KDtree, also known as k-d tree, to find all stars interacting within a given radius (say two light years) due to the collision of two galaxies - This has been moved to an separate optional notebook in the current folder.

## Library Dependencies:

 - **pip install pickle**
 - also requires these libraries if they are not already installed: **matplotlib, numpy, pandas, time, sklearn**
 
To synthesized the galaxy data uses a parametric equations described in the following paper [A New Formula Describing the Scaffold Structure of Spiral Galaxies](https://arxiv.org/ftp/arxiv/papers/0908/0908.0892.pdf) regarding the parametric equation of arm: :


$$ r \left( \phi \right) = \frac{A}{log(B \  \ tan(\frac{\phi}{2N}))  } $$

The synthesizer used here, generates an arbitrary number of arms, generates a 3D gauassian distribution of stars around a galactic center, then distributes a gausian distribiutoin  of stars along the length of each arm. In addition, it generates an arbitrary number of "globular clusters" of stars accoring to a 3D gaussian distribution sprinkled out randomly along the arm curves.

We also used rotation matrices  from this blog [3D Rotations and Euler angles in Python](https://www.meccanismocomplesso.org/en/3d-rotations-and-euler-angles-in-python/)


# Fictitious Galaxies: 

We create two fictitious galaxies: GFFA ("galaxy far, far away") and Xeno (The purported galaxy for Superman's planet Krypton). We intersect the galaxies and use various classification algorithms to identify the stars in each galaxy.

# k Nearest Neighbors

kNN classification follows the general workflow described in the oneAPI Giyhub repository [Classification Usage Model](https://oneapi-src.github.io/oneDAL/daal/usage/training-and-prediction/classification.html#classification-usage-model).

k-Nearest Neighbors (kNN) classification is a classification (or regression) algorithm. The model of the kNN classifier is based on feature vectors and class labels from the training data set. It uses distances between points as a key element for classifying similar points.

# Random Forest

Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks that operates by constructing a multitude of decision trees at training time. For classification tasks, the output of the random forest is the class selected by most trees.

# Define GPU Context and Fit Multiple Sklearn Models Using GPU

We try muliple models and multiple training size values to determine which comboination of models and training size yield optimimum Receiver Operator Characteristic (ROC) scores

In [ ]:
#%load 04_analyzeGalaxyBatch.py

In [ ]:
# %load 04_analyzeGalaxyBatch.py
import warnings
warnings.filterwarnings("ignore", message="A column-vector y was passed when a 1d array was expected")

from sklearnex import patch_sklearn, unpatch_sklearn
patch_sklearn()

#from sklearn import datasets
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.svm import SVC
#from sklearn.svm import NuSVC
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import time
#from random import sample
from sklearn.neighbors import KDTree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def read_dictionary(fn):
    import pickle
    # Load data (deserialize)
    with open(fn, 'rb') as handle:
        dictionary = pickle.load(handle)
    return dictionary

XenoSupermanGalaxy = read_dictionary('XenoSupermanGalaxy.pkl')
GFFA = read_dictionary('GFFA.pkl')

plt.style.use('dark_background')

# dataset is subset of stars from each galaxy
TrainingSize = min(len(GFFA['Stars']), len(XenoSupermanGalaxy['Stars'] ) ) 

collision = dict()
collision['Arms'] = np.vstack((GFFA['Arms'].copy(), XenoSupermanGalaxy['Arms'].copy()))
collision['CenterGlob'] = np.vstack((GFFA['CenterGlob'].copy(), XenoSupermanGalaxy['CenterGlob'].copy()))
collision['Stars'] = np.vstack((GFFA['Stars'].copy(), XenoSupermanGalaxy['Stars'].copy()))
collision['Stars'].shape

# get the index of the stars to use from XenoSupermanGalaxy
XenoIndex = np.random.choice(len(XenoSupermanGalaxy['Stars']), TrainingSize, replace=False)
# get the index of the stars to use from GFFAIndex
GFFAIndex = np.random.choice(len(GFFA['Stars']), TrainingSize, replace=False)

# create a list with a labelforeahc item in the combined training set
# the first hald of the list indicates that class 0 will be for GFFA, 1 will be XenoSupermanGalaxy
y = [0]*TrainingSize + [1]*TrainingSize
# Stack the stars subset in same order as the labels, GFFA first, XenoSupermanGalaxy second
trainGalaxy = np.vstack((GFFA['Stars'][GFFAIndex], XenoSupermanGalaxy['Stars'][XenoIndex]))  

x_train, x_test, y_train, y_test = train_test_split(trainGalaxy, np.array(y), train_size=0.05)

# determine if GPU available:
gpu_available = False
cpu_available = False
dpctl_available = False
try:
    # modern approach for SYCL context is to use dpctl & sklearnex._config supported in 
    # Intel(R) Extension for Scikit-learn* 2021.4 
    import dpctl
    from sklearnex._config import config_context
    dpctl_available = True
except ImportError:
    try:
        # older approach for SYCL context is to use dall4py  
        from daal4py.oneapi import sycl_context
        print("*" * 80)
        print("\ndpctl package not found, switched to daal4py package\n")
        print("*" * 80)
    except ImportError:
        print("\nRequired packages not found, aborting...\n")
        exit()

gpu_available = False
if not dpctl_available:
    try:
        with sycl_context('gpu'):
            gpu_available = True
            def get_context(device):
                 return sycl_context(device)
    except Exception:
        gpu_available = False
else:
    try:
        with config_context(target_offload='gpu'):
            gpu_available = True
            def get_context(device):
                return config_context(target_offload='gpu')
    except Exception:
        gpu_available = False
print('dpctl_available: ', dpctl_available)               
print('gpu_available: ', gpu_available)
print('cpu_available: ', cpu_available)

K = 3
myModels = {'KNeighborsClassifier':KNeighborsClassifier(n_neighbors = K) , 
            'RandomForestClassifier': RandomForestClassifier(n_jobs=2, random_state=0), 
            #'SVC':SVC(gamma='auto'), 
           }
TrainingSize = [.001, .01, .05, .1, .8]
bestScore = {}
hi = 0
K = 3
       
for tsz in TrainingSize:
    x_train, x_test, y_train, y_test = train_test_split( \
                trainGalaxy, np.array(y), train_size=tsz)
    y_train = y_train.ravel()
    y_test = y_test.ravel()

    for name, modelFunc in myModels.items():       
        start = time.time()
        model = modelFunc
        if gpu_available:
            with get_context('gpu'):
                print('Running on gpu')
                model.fit(x_train, y_train)
                y_pred = model.predict(x_test)
        else:
            print('Running on login')
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test) 
                
        print('Display results of {} classification'.format(name))
        print('  K: ',K)
        print('  Training size: ', tsz)
        print('  y_train.shape: ',y_train.shape)
        roc = roc_auc_score(y_test, y_pred)
        print('  roc_auc_score: {:4.1f}'.format(100*roc))
        print('  Time: {:5.1f} sec\n'.format( time.time() - start))
        if roc > hi:
            hi = roc
            bestScore = {'name': name,
                    'roc':roc, 
                    'trainingSize':tsz, 
                    'confusionMatrix': confusion_matrix(y_test, y_pred), 
                    'precision': 100*precision_score(y_test, y_pred, average='binary'),
                    'recall': recall_score(y_test, y_pred, average='binary') }
print('bestScore: name', bestScore['name'])
print('bestScore: confusion Matrix', bestScore['confusionMatrix'])
print('bestScore: precision', bestScore['precision'])
print('bestScore: recall', bestScore['recall'])
print('bestScore: roc', bestScore['roc'])

# Notices & Disclaimers

# Intel technologies may require enabled hardware, software or service activation.

# No product or component can be absolutely secure.
# Your costs and results may vary.

# © Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. 
# *Other names and brands may be claimed as the property of others.


In [ ]:
#!python 04_analyzeGalaxyBatch.py   # works on Windows
! chmod 755 q; chmod 755 run_ModelCompare.sh; if [ -x "$(command -v qsub)" ]; then ./q run_ModelCompare.sh; else ./run_ModelCompare.sh; fi

# Plot result of classification

This time a static image of a previous run of the algorithm showing one galaxy edge on

![galaxyRed_BlueEdge.png](Assets/galaxyRed_BlueEdge.png)


# Notices & Disclaimers

Intel technologies may require enabled hardware, software or service activation.

No product or component can be absolutely secure.
Your costs and results may vary.

© Intel Corporation. Intel, the Intel logo, and other Intel marks are trademarks of Intel Corporation or its subsidiaries. 
*Other names and brands may be claimed as the property of others.
